In [1]:
import requests
import csv
import os
import urllib
import sys
import importlib
import pandas as pd
from shapely.geometry import Point, Polygon
import time
import datetime
from datetime import datetime
from time import sleep
import geopandas 

sys.path.insert(1, '../utils')

import utils

In [2]:
s = requests.Session()

In [3]:
importlib.reload(utils)

<module 'utils' from '../utils/utils.py'>

In [4]:
region = 'New York'
extent = 'region'
county_ids = utils.county_ids.get_county_ids(region, extent)
input_date = '2020-05-01'
input_ts = time.mktime(datetime.strptime(input_date, "%Y-%m-%d").timetuple())
xmin, xmax, ymin, ymax = utils.geometry.osm_bounds(region, county_ids, extent, file = False, raw = True)

url = 'https://api.transitfeeds.com'

In [5]:
# bounding box

coords = [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)]
poly = Polygon(coords)

In [6]:
# api key

#key = '9efe3e4d-9b04-467c-9c72-0cc3cf666fd0'
key = 'f2a91a7e-154d-434a-8083-2cd18e25f3d2'

In [7]:
output_feed_info = [list(['operator_name', 'operator_website', 'operator_location', 'operator_metro', 'feed_id', 'date_query', 
                         'date_fetched', 'earliest_calendar_date', 'latest_calendar_date', 'transit_feeds_url'])]

In [8]:
# gets all available locations on transit feeds

response = s.get(
    url+'/v1/getLocations',
    params = {'key': key
       
    }
)
locations = response.json()['results']['locations']
sleep(15)

In [9]:
# filters locations to only those within bounding box

location_ids = []
for operator in locations:
    pt = Point(operator['lng'], operator['lat'])
    if pt.within(poly) is True: 
        location_ids.append(operator['id'])

In [10]:
# loops through all locations
error_count = 0
id_lst = []
for ids in location_ids:
    for attempt in range(4):
        try: 
            response = s.get(
                url+'/v1/getFeeds',
                params = {'key': key, 'location' : ids, 'type': 'gtfs', 'limit': 200

                }
            )
            feeds = response.json()['results']['feeds']
            id_lst.append(feeds[0]['id'])
            sleep(1)
            print(feeds[0]['id'])
            break
        except:
            print(attempt)
            sleep(60)


id_lst

greater-bridgeport-transit/1022
mta/81
mta/80
port-authority-of-new-york-and-new-jersey/384
suffolk-county-transit/1121
0
1


KeyboardInterrupt: 

In [12]:
id_lst


['mta/81',
 'mta/80',
 'port-authority-of-new-york-and-new-jersey/384',
 'nyc-ferry/944',
 'jfk-airtrain/433',
 'mta/84']

In [25]:
for row in id_lst:
    print(row)
    for attempt in range(5):

        try:
            response = s.get(
                url+'/v1/getFeedVersions',
                params = {'key': key, 'feed' : row, 'limit' : 999

                }
            )
            feed_info = response.json()['results']['versions']
            for version in feed_info:

            # checks latest version to input date

                if input_ts > version['ts']:

                    # writes table for metadata

                    op_name = version['f']['t']
                    op_url = version['f']['u']['i']
                    op_loc = version['f']['l']['t']
                    op_id = version['f']['id']
                    date_fetched = str(datetime.utcfromtimestamp(version['ts']).strftime('%Y-%m-%d'))
                    earliest_calendar_date = str(datetime.strptime(version['d']['s'], '%Y%m%d').date())
                    latest_calendar_date =  str(datetime.strptime(version['d']['f'], '%Y%m%d').date()) 
                    url = version['url']
                    temp = list([op_name, op_url, op_loc, region, op_id, input_date, date_fetched, earliest_calendar_date,
                               latest_calendar_date, url])
                    output_feed_info.append(temp)

                    # downloads gtfs

                    urllib.request.urlretrieve(feed_info[1]['url'], "../gtfs/feeds_" + input_date + "/" + op_name + '-' + input_date + ".zip")

            break
        except:
            error_count = attempt
            if attempt <3:
                sleep(30)
                print(attempt)
            elif attempt < 5:
                sleep(60)
                print(attempt)
            else: 
                break

   

mta/81
0
1
2
3
4
mta/80
0
1
2
3
4
port-authority-of-new-york-and-new-jersey/384
0
1
2
3
4
nyc-ferry/944
0
1
2
3
4
jfk-airtrain/433
0
1
2
3
4
mta/84
0
1
2
3
4


In [24]:
output_feed_info




[['operator_name',
  'operator_website',
  'operator_location',
  'operator_metro',
  'feed_id',
  'date_query',
  'date_fetched',
  'earliest_calendar_date',
  'latest_calendar_date',
  'transit_feeds_url'],
 ['MTA Bronx GTFS',
  'http://mta.info/developers/download.html',
  'Bronx, NY, USA',
  'New York',
  'mta/81',
  '2020-05-01',
  '2020-04-04',
  '2020-01-18',
  '2020-05-02',
  'https://transitfeeds.com/p/mta/81/20200403/download'],
 ['MTA Bronx GTFS',
  'http://mta.info/developers/download.html',
  'Bronx, NY, USA',
  'New York',
  'mta/81',
  '2020-05-01',
  '2020-04-04',
  '2020-01-18',
  '2020-05-02',
  'https://transitfeeds.com/p/mta/81/20200403/download'],
 ['MTA Bronx GTFS',
  'http://mta.info/developers/download.html',
  'Bronx, NY, USA',
  'New York',
  'mta/81',
  '2020-05-01',
  '2020-04-04',
  '2020-01-18',
  '2020-05-02',
  'https://transitfeeds.com/p/mta/81/20200403/download'],
 ['MTA Bronx GTFS',
  'http://mta.info/developers/download.html',
  'Bronx, NY, USA',
  '